In [ ]:
# quiver 绘制向量场
import matplotlib.pyplot as plt
import numpy as np

# 定义 x 和 y 的网格坐标
x, y = np.meshgrid(np.arange(0, 10), np.arange(0, 10))

# 定义向量的 x 和 y 分量
u = np.sin(x)
v = np.cos(y)

# 绘制向量场
plt.quiver(x, y, u, v)

# 显示图形
plt.show()

In [ ]:
import numpy as np

class ThreeNeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # 随机初始化权重参数
        self.W1 = np.random.randn(self.input_size, self.hidden_size)
        self.b1 = np.random.randn(self.hidden_size)
        self.W2 = np.random.randn(self.hidden_size, self.output_size)
        self.b2 = np.random.randn(self.output_size)
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_grad(self, x):
        return (1.0 - self.sigmoid(x)) * self.sigmoid(x)
    
    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=-1, keepdims=True)
    
    def forward(self, x):
        # 第一层
        a1 = np.dot(x, self.W1) + self.b1
        z1 = self.sigmoid(a1)
        
        # 第二层
        a2 = np.dot(z1, self.W2) + self.b2
        y = self.softmax(a2)
        
        return y
    # 交叉熵函数
    def loss_v0(self, x, t):
        y = self.forward(x)
        loss = -np.sum(t*np.log(y+1e-7)) / x.shape[0]
        
        return loss
    
    # 均方差函数
    def loss(self, x, t):
        y = self.forward(x)
        loss = np.sum((y - t)**2) / (2 * x.shape[0])
    
        return loss
    
    def accuracy(self, x, t):
        y = self.forward(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def train(self, x, t, learning_rate=0.1):
        batch_num = x.shape[0]
        # 前向传播
        a1 = np.dot(x, self.W1) + self.b1
        z1 = self.sigmoid(a1)
        a2 = np.dot(z1, self.W2) + self.b2
        y = self.softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grad_W2 = np.dot(z1.T, dy)
        grad_b2 = np.sum(dy, axis=0)
        
        dz1 = np.dot(dy, self.W2.T)
        da1 = self.sigmoid_grad(a1) * dz1
        grad_W1 = np.dot(x.T, da1)
        grad_b1 = np.sum(da1, axis=0)
        
        # 更新参数
        self.W1 -= learning_rate * grad_W1
        self.b1 -= learning_rate * grad_b1
        self.W2 -= learning_rate * grad_W2
        self.b2 -= learning_rate * grad_b2

In [ ]:
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = ThreeNeuralNetwork(input_size=784, hidden_size=100, output_size=10)

iters_num = 10000  # 繰り返しの回数を適宜設定する
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    network.train(x_batch, t_batch)
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

# グラフの描画
markers = {'train': 'o', 'test': 's'}
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, label='train acc')
plt.plot(x, test_acc_list, label='test acc', linestyle='--')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()